In [63]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\ACER\.cache\kagglehub\datasets\jessicali9530\stanford-dogs-dataset\versions\2


In [137]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from sklearn import preprocessing

In [138]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [139]:
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform

        # Loop through each folder (class) and collect image paths and labels
        self.classes = os.listdir(image_dir)
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        for cls in self.classes:
            class_folder = os.path.join(image_dir, cls)
            if os.path.isdir(class_folder):
                for img_name in os.listdir(class_folder):
                    img_path = os.path.join(class_folder, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[cls])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        img = Image.open(img_path)  # Open image

        if img.mode == 'RGBA':
            img = img.convert('RGB')  # Convert RGBA to RGB
        if self.transform:
            img = self.transform(img)  # Apply transformations
        return img, label


In [140]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=7, padding=4, bias=False)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=7, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout(0.5)
        # Define a pooling layer, e.g., MaxPool2d
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Added pooling layer

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.softmax(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [141]:
image_dir = 'D:/Kerja/Dataset/images/Images'

In [142]:
dataset = CustomImageDataset(image_dir=image_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [154]:
class_labels = list(dataset.class_to_idx.keys())
print("Class labels:", class_labels)

Class labels: ['affenpinscher', 'Afghan hound', 'African hunting dog', 'Airedale', 'American Staffordshire terrier', 'Appenzeller', 'Australian terrier', 'basenji', 'basset', 'beagle', 'Bedlington terrier', 'Bernese mountain dog', 'black and tan coonhound', 'Blenheim spaniel', 'bloodhound', 'bluetick', 'Border collie', 'Border terrier', 'borzoi', 'Boston bull', 'Bouvier des Flandres', 'boxer', 'Brabancon griffon', 'briard', 'Brittany spaniel', 'bull mastiff', 'cairn', 'Cardigan', 'Chesapeake Bay retriever', 'Chihuahua', 'chow', 'clumber', 'cocker spaniel', 'collie', 'curly-coated retriever', 'Dandie Dinmont', 'dhole', 'dingo', 'Doberman', 'English foxhound', 'English setter', 'English springer', 'EntleBucher', 'Eskimo dog', 'flat-coated retriever', 'French bulldog', 'German shepherd', 'German short-haired pointer', 'giant schnauzer', 'golden retriever', 'Gordon setter', 'Great Dane', 'Great Pyrenees', 'Greater Swiss Mountain dog', 'groenendael', 'Ibizan hound', 'Irish setter', 'Irish t

In [143]:
from torchvision.models import resnet18

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(dataset.classes)  # Number of classes (subfolders)
model = resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [144]:
print("Number of classes:", num_classes)

Number of classes: 120


In [148]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0

    # Iterate over the data
    for images, labels in tqdm(train_loader):
        # Move images and labels to the device (GPU or CPU)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Print statistics
        running_loss += loss.item()

    # Print loss and accuracy for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

100%|██████████| 644/644 [08:16<00:00,  1.30it/s]


Epoch [1/50], Loss: 3.8264, Accuracy: 10.60%


100%|██████████| 644/644 [07:46<00:00,  1.38it/s]


Epoch [2/50], Loss: 3.6811, Accuracy: 13.18%


100%|██████████| 644/644 [07:44<00:00,  1.39it/s]


...

Epoch [48/50], Loss: 0.4477, Accuracy: 86.10%


100%|██████████| 644/644 [07:37<00:00,  1.41it/s]


Epoch [49/50], Loss: 0.4555, Accuracy: 85.84%


100%|██████████| 644/644 [07:37<00:00,  1.41it/s]

Epoch [50/50], Loss: 0.4410, Accuracy: 86.23%


In [149]:
# Save the model's state dictionary
torch.save(model.state_dict(), "model.pth")
print("Model saved successfully!")


Model saved successfully!


In [150]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            # Move data to the device (GPU/CPU)
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)

            # Get predictions
            _, predicted = torch.max(outputs.data, 1)

            # Update total and correct counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = 100 * correct / total
    print(f"Accuracy on test data: {accuracy:.2f}%")
    return accuracy


In [152]:
# Assuming test_loader is defined and device is set
accuracy = evaluate_model(model, train_loader, device)


Evaluating: 100%|██████████| 644/644 [03:28<00:00,  3.08it/s]

Accuracy on test data: 88.65%
